# 3th attempt - NN

In [33]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [34]:
SIZE = 10 
AMOUNT_BOARDS = 100

In [35]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_non-reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [36]:
new_columns = [f'Col_{i}' for i in range(1, gen*SIZE*SIZE + 1)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [37]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 3781
reverse df sort: 3781


In [38]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.1, random_state=365)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

len x train:  3061
len x test:  379
len y train:  3061
len y test:  379


In [39]:
X_train.shape

(3061, 100)

In [40]:
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()

In [41]:
print(X_train_array.shape)
print(y_train_array.shape)

(3061, 100)
(3061,)


In [42]:
X_train_array = X_train_array.reshape((X_train.shape[0],SIZE,SIZE,1))
y_train_array = y_train_array.reshape((y_train.shape[0],1))

In [43]:
print(X_train_array.shape)
print(y_train_array.shape)

(3061, 10, 10, 1)
(3061, 1)


In [44]:
X_val_array = X_val.to_numpy()
X_val_array = X_val_array.reshape((X_val.shape[0],SIZE,SIZE,1))
y_val_array = y_val.to_numpy()
y_val_array = y_val_array.reshape((y_val.shape[0],1))

X_test_array = X_test.to_numpy()
X_test_array = X_test_array.reshape((X_test.shape[0],SIZE,SIZE,1))
y_test_array = y_test.to_numpy()
y_test_array = y_test_array.reshape((y_test.shape[0],1))

In [ ]:
import tensorflow as tf

input_shape = (10, 10, 1)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),  

    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')  
])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,249 (83.00 KB)

 Trainable params: 21,249 (83.00 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [47]:
history = model.fit(X_train_array, y_train_array, 
    validation_data=(X_val_array, y_val_array),
    epochs=10,
    batch_size=32)


Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7592 - loss: 0.5421 - val_accuracy: 0.7830 - val_loss: 0.4285
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7810 - loss: 0.3941 - val_accuracy: 0.8035 - val_loss: 0.3323
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8505 - loss: 0.2790 - val_accuracy: 0.8387 - val_loss: 0.2950
Epoch 4/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9002 - loss: 0.2217 - val_accuracy: 0.8592 - val_loss: 0.2550
Epoch 5/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9367 - loss: 0.1763 - val_accuracy: 0.8856 - val_loss: 0.2269
Epoch 6/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9687 - loss: 0.1123 - val_accuracy: 0.9179 - val_loss: 0.1866
Epoch 7/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9857 - loss: 0.0764 - val_accuracy: 0.9120 - val_loss: 0.1784
Epoch 8/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9945 - loss: 0.0471 - val_accuracy: 0.9267 - val_loss:

In [48]:
test_loss, test_acc = model.evaluate(X_test_array, y_test_array)
print(f"Test accuracy: {test_acc:.3f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9229 - loss: 0.1689 
Test accuracy: 0.921
